In [1]:
import pandas as pd
import numpy as np 
from pandas.tseries.offsets import BDay

In [2]:
%%time

# read OCP IA: if India Time on or before 3PM, then read EUMOCP else AMMOCP
# This needs to be changed - script should run as soon the new file is available
import pytz
from datetime import datetime, timedelta,date
d = datetime.now(pytz.timezone('Asia/Calcutta'))
if d.hour <= 15:
    file = 'OCP_IA_EUMOCP'
else:
    file = 'OCP_IA_AMMOCP'
    
d = d.strftime('%d-%b-%Y').upper()

#path = r'\\susday014.prod.local\cdunix\ERP\RR\Inbound\OCP\IA\rcv'
path = r'E:\_Projects\orders\rcv'
# file = 'OCP_IA_EUMOCP'
file = 'OCP_IA_EUMOCP'

#New Script
%run Orders.ipynb
d = EUMOCP_File_Name

#Old
#d = '29-FEB-2024'

data = pd.read_excel(path + '\\' + file + '_' + d + '.xlsx', sheet_name='Sheet1', engine='openpyxl', \
                     na_values='', keep_default_na=True)

print(file + '_' + d + '.xlsx')

OCP_IA_EUMOCP_29-FEB-2024.xlsx
CPU times: total: 20.4 s
Wall time: 46.6 s


In [3]:
data_bkp = data.copy()
# data = data_bkp.copy()


In [4]:
# Exclude ERP LINE STATUS = Closed (Updated dt 12-Jun-2023)
data = data[data['ERP Line Status'] != 'Closed']

In [5]:
# Update Product group and Offer PF; +2 cols
prod = pd.read_csv(r'E:\_Projects\lookups\class_offerpf_map.csv', \
                   encoding='utf-8', low_memory=False)
data['Class'] = data['Class'].astype(str)
data = pd.merge(data, prod[['class', 'prod_grp_wb','offer_pf_wb','division_wb']], left_on='Class', right_on='class', how='left',validate='m:1')
data[['Products', 'Offer PF','LOB Prod']] = data[['prod_grp_wb', 'offer_pf_wb', 'division_wb']]
data.drop(columns={'class', 'offer_pf_wb','prod_grp_wb','division_wb'}, axis=1, inplace=True)   

In [6]:
# Update MCID TAGGING name from RACE; +1 col
tag = pd.read_excel(r'E:\_Projects\lookups\MCID_Tagging _Master.xlsx', sheet_name='MCID_Tagging__Master_TABLE' \
                   , engine='openpyxl')
tag.drop_duplicates(inplace=True,subset=['MCID'],keep='first')
data = pd.merge(data, tag[['MCID','Tag Name']], left_on='PID', right_on='MCID', how='left',validate='m:1')
data.drop(columns={'MCID'}, axis=1, inplace=True)   
data['Tag Name'] = np.where(data['Tag Name'].isnull(),data['PID Desc'],data['Tag Name'])

In [7]:
# Pref offer
pref_offer = pd.read_csv(r'E:\_Projects\lookups\preferred_offer.csv', \
                    encoding='utf-8', low_memory=False)
pref_offer_grpd = pref_offer.groupby('PID')['Type'].max()
data = pd.merge(data, pref_offer_grpd, left_on='PID', right_on='PID', how='left',validate='m:1')
data.rename(columns={'Type' : 'Preferred Offer'}, inplace=True)

In [8]:
# Prod Cate: 7360 category as Std, Narrow Core, Jarvis; 2012 vs 2021
# condtn?
data['Prod Cate'] = ''

In [9]:
#Update DOM / EXPORT for chennai
# Delete SS21, Dom/Export and start using Prod Cate itself
# 
df_dom_exp = pd.read_csv(r'E:\_Projects\lookups\bom_details.csv', \
                        encoding='utf-8', low_memory=False)
df_dom_exp_grpd = df_dom_exp.groupby('mcid')[['Dom/Export']].max()
data = pd.merge(data, df_dom_exp_grpd, left_on='PID', right_on='mcid', how='left',validate='m:1')

data['Prod Cate'] = np.where( (data['Dom/Export'] != '') & (data['Whse'].isin(['CHE','CHS'])), \
                                    data['Class'] + data['Dom/Export'].str[:1], '')
# data['Prod Cate'] = np.where(data['Prod Cate'] != '' & data['Dom/Export'] != '', \
#                              data['Prod Cate'] + ';' + data['Dom/Export'], \
#                              data['Dom/Export'])
data.drop(columns={'Dom/Export'}, inplace=True)

In [10]:
# based on PID Desc like SelfServ*21* 
# based on TAG Name like *SS21*
data['IsSS21']=''
data['IsSS21'] = np.where((data['Class']=='2012') & (data['PID Desc'].str.contains('&'.join(['SelfServ','21']))==True), \
                          'SS21', '')
data['IsSS21'] = np.where((data['Class']=='2012') & (data['Tag Name'].str.endswith('SS21')==True), 'SS21', data['IsSS21'])
data['Prod Cate'] = np.where(data['Prod Cate'].notnull() & data['IsSS21'].notnull(), data['Prod Cate']+';'+data['IsSS21'], \
                            data['IsSS21'])
data.drop(columns={'IsSS21'}, inplace=True)

In [11]:
# Add PLANT field

incl_type = ['PO','PD','Planned Buy', 'Planned Make','Planned order','Flow']
pid_type = ['M','K','C']
data['Plant'] = ''
conditions = [ (data['Demand']!='WIP OSP') & data['Whse'].isin(['CHE','CHS']) , \
             (data['Demand']!='WIP OSP') &  \
                  data['Supplier'].isin(['JABIL CIRCUIT INC - CHIHUAHUA']) , \
             (data['Demand']!='WIP OSP') & data['Whse'].isin(['USL']) & \
                  ~data['Supplier'].isin(['UNIVERSAL SCIENTIFIC INDUSTRIAL DE MEXICO S.A. DE C.V']) , \
             (data['Demand']!='WIP OSP') & data['Whse'].isin(['US','USL','GF1'])  & \
                  data['Supplier'].isin(['UNIVERSAL SCIENTIFIC INDUSTRIAL DE MEXICO S.A. DE C.V']) , \
             (data['Demand']!='WIP OSP') & \
                  data['Supplier'].isin(['ENNOCONN HUNGARY KFT'])
             ]
choices = ['CHE','JAB','USL','USI','ENN']
data['Plant'] = np.select(conditions, choices, default='')

In [12]:
## PO Number update for ENN, JAB, USI ************************************************************************
lst_splr = ['JABIL CIRCUIT INC - CHIHUAHUA', 'UNIVERSAL SCIENTIFIC INDUSTRIAL DE MEXICO S.A. DE C.V', \
                'UNIVERSAL SCIENTIFIC INDUSTRIAL DE MEXICO S.A. DE C.V', \
                'ENNOCONN HUNGARY KFT']
lst_type = ['PD','PO']

condtn1 = data['Supplier'].isin(lst_splr)
condtn3 = data['Supply Type'].isin(lst_type)

 # Trim Supply Detail (PO) for OCP-IA records
data['Supply PO'] = np.where(condtn1 & condtn3, \
                                   data['Supply Detail'].str.split('(').str[0].copy(), '')

# bring in IO/PO field
data['IO/PO No.'] = np.where(data['Supply PO']!='', data['Supply PO'], data['IO No.'])

data['PO No.'] = data['Supply PO']
data.drop(columns={'Supply PO'}, inplace=True)

# col = data.pop('IO/PO No.')
# data.insert(33, col.name, col)

In [13]:
#Formatting DATE fields
data['Booked Dt'] = pd.to_datetime(data['Booked Dt'], format='%Y-%m-%d')
data['Request Dt'] = pd.to_datetime(data['Request Dt'], format='%Y-%m-%d', errors='coerce')
data['PD_Order'] = pd.to_datetime(data['PD_Order'], format='%Y-%m-%d', errors='coerce')
data['Ship Date'] = pd.to_datetime(data['Ship Date'], format='%Y-%m-%d')
data['SAD'] = pd.to_datetime(data['SAD'], format='%Y-%m-%d')

# data['IO BOOKED DATE'] = pd.to_datetime(data['IO BOOKED DATE'], format='%d-%m-%Y %I:%M:%S %p')
data['IO BOOKED DATE'] = pd.to_datetime(data['IO BOOKED DATE'], format='%Y-%m-%d')
data['IO BOOKED DATE'] = data['IO BOOKED DATE'].dt.date
data['IO SSD'] = pd.to_datetime(data['IO SSD'], format='%Y-%m-%d')
data['IO SAD'] = pd.to_datetime(data['IO SAD'], format='%Y-%m-%d')

data['PD_PO'] = pd.to_datetime(data['PD_PO'], format='%Y-%m-%d')
data['CRSD'] = pd.to_datetime(data['CRSD'], format='%Y-%m-%d')

#data['PRD'] = pd.to_datetime(data['PRD'], format='mixed').dt.date()
data['PRD'] = pd.to_datetime(data['PRD'], format='%Y-%m-%d')
# data['PRD'] = pd.to_datetime(data['PRD'], format='%d-%m-%Y')
data['Plant Ship date'] = pd.to_datetime(data['Plant Ship date'], format='%Y-%m-%d')
data['NEED BY DATE'] = pd.to_datetime(data['NEED BY DATE'], format='%Y-%m-%d')

In [14]:
# #************************ Update IO/PO SSD *************************************
data['IO/PO SSD'] = ''
data['IO/PO SSD'] = np.where(data['Supplier'].isin(lst_splr) \
                             & data['PD_PO'].notnull() & ~data['Plant'].isin(['']), \
                             data['PD_PO'], data['IO/PO SSD'])

data['IO/PO SSD'] = np.where(data['Supplier'].isin(lst_splr) \
                             & data['NEED BY DATE'].notnull() & data['PD_PO'].isnull() & ~data['Plant'].isin(['']), \
                             data['NEED BY DATE'], data['IO/PO SSD'])

data['IO/PO SSD'] = np.where(data['Supplier'].isin(lst_splr) \
                             & data['IO SSD'].notnull() & data['PD_PO'].isnull() & data['NEED BY DATE'].isnull() & ~data['Plant'].isin(['']), \
                             data['IO SSD'], data['IO/PO SSD'])


# #***** LESS transit days for JABIL: 3 days
data['IO/PO SSD'] = np.where((data['Supplier']=='JABIL CIRCUIT INC - CHIHUAHUA') \
                             & ~data['IO/PO SSD'].isin(['']) &data['PD_PO'].notnull() & ~data['Plant'].isin(['']), \
                             pd.to_datetime(data['IO/PO SSD']) - BDay(3), data['IO/PO SSD'])

#******** less Transit days for USI: 5 days
data['IO/PO SSD'] = np.where(~data['IO/PO SSD'].isin(['']) &data['PD_PO'].notnull() & data['Plant'].isin(['USI']), \
                             pd.to_datetime(data['IO/PO SSD']) - BDay(5), data['IO/PO SSD'])



# #*************** Update Plant SSD as IO SSD else Ship Date (for OTHER THAN jabil, ennoconn, usi)
data['IO/PO SSD'] = np.where((~data['Supplier'].isin(lst_splr)) \
                             & data['IO SSD'].notnull() & ~data['Plant'].isin(['']), \
                             data['IO SSD'], data['IO/PO SSD'])

data['IO/PO SSD'] = pd.to_datetime(data['IO/PO SSD'], format='%Y-%m-%d')


In [15]:
# #************************ Update Plant SSD *************************************
cond = (data['IO/PO SSD'].isin(['']) | data['IO/PO SSD'].isnull()) & ~(data['Plant'].isin(['']))
data['Plant SSD'] = np.where(cond,data['Ship Date'],data['IO/PO SSD'])
data['Plant SSD'] = pd.to_datetime(data['Plant SSD'], format='%Y-%m-%d')

In [16]:
#func to popualte WK, MON, QTR
def upd_period(df, fld, fld_wk, fld_mon, fld_qtr):
    df[fld_wk] = pd.to_datetime(df[fld], errors='coerce').dt.isocalendar().week.astype(pd.Int64Dtype())
    df[fld_mon] = pd.to_datetime(df[fld], errors='coerce').dt.strftime('%Y%m')
    df[fld_qtr] = pd.to_datetime(df[fld], errors='coerce').dt.strftime('%Y') + 'Q' + \
                        + pd.to_datetime(data[fld], errors='coerce').dt.quarter.astype(pd.Int64Dtype()).astype(str)

In [17]:
#align PAST dates into last week 
#Sri made changes on 07-Aug-23
def align_past(fld):
    data[fld] = data[fld].astype('datetime64[ns]')
    pre_week = date.today() - timedelta(days=1)
    #data['Previous_week_Date'] = pre_week- timedelta(days = pre_week.weekday())
    data['Previous_week_Date'] = pre_week
    data['Previous_week_Date'] = data['Previous_week_Date'].astype('datetime64[ns]')
#     data['final'] =  data[fld]
#     cond = data['final'] < datetime.today()
    cond = data[fld] < pd.to_datetime('today')
    data[fld] = np.where(cond, data['Previous_week_Date'], data[fld])
    data[fld] = data[fld].astype('datetime64[ns]')
    
#align_past('Ship Date')
upd_period(data, 'Ship Date', 'Ship Dt Wk', 'Ship Dt Mon', 'Ship Dt Qtr')
#data.drop(columns={'Previous_week_Date'}, axis=1, inplace=True)

#align_past('IO SSD')
upd_period(data, 'IO SSD', 'IO SSD Wk', 'IO SSD Mon', 'IO SSD Qtr')
#data.drop(columns={'Previous_week_Date'}, axis=1, inplace=True)

align_past('IO/PO SSD')
upd_period(data, 'IO/PO SSD', 'IO/PO SSD Wk', 'IO/PO SSD Mon', 'IO/PO SSD Qtr')
data.drop(columns={'Previous_week_Date'}, axis=1, inplace=True)

align_past('Plant SSD')
upd_period(data, 'Plant SSD', 'Plant SSD Wk', 'Plant SSD Mon', 'Plant SSD Qtr')
data.drop(columns={'Previous_week_Date'}, axis=1, inplace=True)
           
upd_period(data, 'PRD', 'PRD Wk', 'PRM', 'PRQ')
data.drop(columns={'PRD Wk'}, axis=1, inplace=True)


In [18]:
#Hari changed New PRD GRP formulae 28-06-2023
def update_prd_group(fld):
    cond = (data[fld]!= '') & (data[fld].astype(str) == data['PRQ'].astype(str))
    cond1 =  (data[fld]!= '') & (data['PRQ'].astype(str) > data[fld].astype(str))
    cond2 =  (data[fld]!= '') & (data['PRQ'].astype(str) < data[fld].astype(str))
    data['PRD Grp'] = np.where(cond,'PRD Curr', np.where(cond1,'PRD Fut', np.where(cond2,'PRD Past','')))
    

In [19]:
data['PRD Grp']=''
data['Plant SSD Qtr'] = data['Plant SSD Qtr'].replace(np.nan, '', regex=True)
data['Ship Dt Qtr'] = data['Ship Dt Qtr'].replace(np.nan, '', regex=True)
update_prd_group('Plant SSD Qtr')
#update_prd_group('Ship Dt Qtr')

In [20]:
cust = pd.read_excel(r'E:\_Projects\lookups\7.3_Country-Key_Account_Heirarchy.xlsb', \
                     sheet_name='Report1', engine='pyxlsb')
cust['Master Customer Number'] = cust['Master Customer Number'].astype(str)
cust_grpd = cust.groupby(['Master Customer Number'])['Master Customer Name'].max()
data = pd.merge(data, cust_grpd, left_on='Master Customer Number', right_on='Master Customer Number', how='left',validate='m:1')
keyacc_grpd = cust.groupby(['Master Customer Number'])['Key Account'].max()
data = pd.merge(data, keyacc_grpd, left_on='Master Customer Number', right_on='Master Customer Number', how='left',validate='m:1')

In [21]:
# Fixing Junk chars in CUST; and 25 chars only
#batch1
data['junk'] = data['Customer_End Customer'].str.len() - \
    data['Customer_End Customer'].str.count(r'[a-zA-Z0-9.,:;\'\"\`~!@#$%^& -_-()/\\]')

data['Customer'] = np.where((data['junk']  <= 3) & (data['Customer_End Customer'] != ''), \
                                 data['Customer_End Customer'], data['Master Customer Name'])

data['Customer'] = np.where((data['Customer'] == 'OTHER') | (data['Customer'].isnull()), \
                               data['Org Code']+' CUSTOMER', data['Customer'])
#batch2
data['junk'] = data['Customer'].str.len() - \
    data['Customer'].str.count(r'[a-zA-Z0-9.,:;\'\"\`~!@#$%^& -_-()/\\]')
data['Customer'] = np.where((data['junk']  > 3), \
                               data['Org Code']+' CUSTOMER', data['Customer'])

data.drop(columns={'Master Customer Name','junk'}, inplace=True, axis=1)

data['Customer'] = np.where(data['Customer'].str.len()>25, data['Customer'].str[:23] + '..', data['Customer'])

In [22]:
data['DEMAND VAL'] = data['DEMAND VAL'].astype('Int64')
data['LN'] = data['Line Number'].astype(str).replace(r'\.', '_', regex=True)
cols_risk2 = ['Org Code','DEMAND VAL','LN','PID']
data['id2'] = data[cols_risk2].astype(str).apply('_'.join, axis=1)

risk = pd.read_csv(r'E:\_Projects\orders\rcv\\risk_upside.csv', encoding='utf-8', low_memory=False)
risk['order_nbr'] = risk['order_nbr'].astype ("Int64").astype(str)
risk['order_line_nbr'] = risk['order_line_nbr'].astype(int).astype(str)
risk['shipment_nbr'] = risk['shipment_nbr'].astype(int).astype(str)

cols_risk1 = ['Country','order_nbr','order_line_nbr','shipment_nbr','item_hyp']
risk['id'] = risk[cols_risk1].astype(str).apply('_'.join, axis=1)
risk.drop_duplicates(inplace=True,subset=['id'],keep='first')
data = pd.merge(data, risk[['id', 'RiskCode', 'RiskMonth', 'UpsideCode', 'UpsideMonth']], \
                left_on=['id2'], right_on=['id'], how='left',validate='m:1')
data.drop(columns={'id','id2'}, axis=1, inplace=True) 

data.drop(columns={'LN'}, axis=1, inplace=True)

In [23]:
#ramiya upside Qtr ---> upside Mthn
upd_period(data, 'UpsideMonth', 'UpsideMonth Wk', 'UpsideMonth Mon', 'UpsideMonth Qtr')
data.drop(columns={'UpsideMonth Wk', 'UpsideMonth Mon'}, axis=1, inplace=True)
print(data['UpsideMonth Qtr'].unique())

[nan '1970Q1']


In [24]:
aas = pd.read_excel(r'E:\_Projects\lookups\aas_logic.xlsx', sheet_name='logic', engine='openpyxl')

data['As_a_Serv'] = ''
for index, row in aas.iterrows():
    data['As_a_Serv'] = np.where((data[row['Field']].str.startswith(row['Customer_AAS'])) & \
                                 (data['PID Type'] == row['PID Type']) & (data['Products'] == row['Prod']), \
                                 'As-a-Serv', data['As_a_Serv'])

In [25]:
# Create UNIQE ID and COMMENTS fields
# data['UID'] = data['Whse'].astype(str) + data['SO Ship Org'].astype(str) + data['Org Code'].astype(str) + \
#             data['DEMAND VAL'].astype(str)  + data['Line Number'].astype(str) + data['PID'].astype(str) + \
#             data['IO/PO No.'].astype(str)
data['Line Number'] = data['Line Number'].fillna('')
data['Line Number'] = data['Line Number'].astype(str)
data['UID'] = data['PID'].astype(str) +'|'+data['Org Code'].astype(str)+'|'+ data['DEMAND VAL'].astype(str) + '|'+data['Line Number'].astype(str)


# col = data.pop('UID')
# data.insert(0, col.name, col)

In [26]:
# data['IO No.'] = data['IO No.'].astype(str)
data['IO/PO No.'] = np.where(data['IO/PO No.'].isnull(),'0',data['IO/PO No.'])
data['IO/PO No.'] = pd.to_numeric(data['IO/PO No.']).astype(int).astype(str)
data['IO/PO No.'] = np.where(data['IO/PO No.']=='0','',data['IO/PO No.'])

data['IID'] =''
data['IID'] = np.where(data['IO/PO No.'].notnull(),data['PID']+'|'+data['IO/PO No.'].astype(str), '')
# col = data.pop('IID')
# data.insert(1, col.name, col)

e:\softwares\anaconda3\Lib\site-packages\pandas\core\dtypes\astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)


In [27]:
#Concate address
data['Ship To Cust'] = data['Ship To Customer Name'] + ', ' + data['Ship To Address1'] +', '+ data['Ship To Address2'] +', '+\
                       data['Ship To City'] + ', ' + data['Ship To State'] +', '+ data['Ship To Zip']
data['Ship to Intmed'] = data['Intmed Ship To'] + ', ' + data['Intmed Ship To Address1'] +', '+ data['Intmed Ship To City'] +', '+\
                       data['Intmed Ship To State'] + ', ' + data['Intmed Ship To Zip'] 
#Drop the column
data.drop(columns={'Ship To Customer Name','Ship To Address1','Ship To Address2','Ship To City','Ship To State','Ship To Zip','Intmed Ship To','Intmed Ship To Address1','Intmed Ship To City','Intmed Ship To State','Intmed Ship To Zip'}, axis=1, inplace=True)

In [28]:
# Qty field - SupplyQty with OH, RS
conditions = [data['Supply Qty'] > 0, data['OH Qty'] > 0, data['Res Qty'] > 0]
choices = [data['Supply Qty'], data['OH Qty'], data['Res Qty']]
data['Qty'] = np.select(conditions, choices)
#Ramya


In [29]:
# _SO_Status
# SO SSD < Today()	SO SSD Past
# SO SAD >= (SO PD and SO PD not null)	SO SSD Late
# Only PBO: SO SSD < Today()+28d	PBO within executn per
# (SO SSD w/o IO/PO AND SO SSD) < Today()+28d	SO w/o Supply
# SO PD is null	SO PD Null
from pandas.tseries.offsets import BDay
lst_sply=['Planned Buy', 'Planned order', 'Purchase requisition']
data['current_date'] = datetime.today()
condtn1=data['Ship Date'] < data['current_date']
condtn2=(data['PD_Order'].notnull()) & (data['SAD'] > data['PD_Order'])

condtn3=(data['Demand']=='Order Pre Bld') & (data['Ship Date'] < (data['current_date']+timedelta(days=28))) & \
            (data['IO/PO No.']!='')

condtn4=(data['Demand']!='Order Pre Bld') & (data['Ship Date'] < (data['current_date']+timedelta(days=28))) & \
            (data['IO/PO No.']=='')  & data['Supply Type'].isin(lst_sply)

condtn5=(data['PD_Order'].isnull() & (data['current_date'] < (data['Booked Dt'] + BDay(5))) & \
             (data['Demand']=='Order'))
condtnX=(~data['Demand'].isin(['Order Flow Stk']) & data['Supply Qty']>0)
condtnY=(data['Supply Qty']>0)

data['_SO_Status'] =''
data['_SO_Status'] = np.where(condtn1 & condtnX, 'SO SSD Past',  \
                              np.where(condtn2 & condtnX,'SO SSD Late', \
                                      np.where((condtn3 & condtnY),'PBO w/n Exec Per', \
                                              np.where(condtn4 & condtnY, 'SO w/o Supply', \
                                                      np.where(condtn5 & condtnY, 'SO PD Null','')))) )

In [30]:
# Read Transit Time
tt = pd.read_csv(r'E:\_Projects\lookups\sit_transit_days.csv', encoding='utf-8', low_memory=False)
data = pd.merge(data, tt, left_on=['Whse','Org Code'], right_on=['SOURCE','CTRY'], how='left',validate='m:1')


In [31]:
# import pyodbc
# Hari/Ramiya: Instead of reading CSV from SUSDAY1322, read directly from MS Acess DB
# E:\RACE\SIT-II\SIT-II.accdb; query is "qry_ship-meth_TT"



In [32]:
# Calc TRANSIT DAYS
# Land --> Land
# Sea/Air --> for all mcids and ATM, SCO, KIOSK products --> Sea
# Sea/Air --> for other products --> Air
data['Transit_recom_mode']=''
condtn1 = data['Default']=='Land'
condtn2 = (data['Default']=='Sea/Air') & (data['Products'].isin(['ATM','SCO','Kiosk'])) & (data['PID Type'].isin(['M','U']))
condtn3 = (data['Default'].notnull()) & (data['Transit_recom_mode']=='')
data['Transit_recom_mode'] = np.where(condtn1, 'Land', \
                                     np.where(condtn2, 'Sea', \
                                             np.where(condtn3, 'Air', '')))
condtns = data['Transit_recom_mode']=='Land', data['Transit_recom_mode']=='Sea', data['Transit_recom_mode']=='Air'
values = data['Land'], data['Sea'], data['Air']
data['Transit_recom_days'] = np.select(condtns, values)

data.drop(columns={'SOURCE','CTRY','Default','Sea','Land','Air'}, inplace=True)

In [33]:
# _IO/PO_Status
# (IO/PO PD/SSD + Transit Time) > SO SSD
# PO PD is null, only if IO SSD + 5 days
# sply_type=['PD','PO','Purchase requisition','InTransit']
# sply_type2=['IREQ','PD','Planned Buy','Planned Order','PO','Purchase requisition']
# # data['IO_PO check'] =''
# # data['IO_PO check'] = np.where(data['PD_PO'].notnull(), data['PD_PO'], \
# #                               np.where(data['IO SSD'].notnull(), data['IO SSD'],data['IO_PO check']))
# # data['IO_PO check'] = pd.to_datetime(data['IO_PO check'])
# data['IO_PO check2'] =  data['Plant SSD'] + pd.to_timedelta(data['Transit_recom_days'], unit='D')

# condtn1a = (data['IO_PO check2'] > data['Ship Date']) & (data['Supply Type']=='PD')
# condtn1b = (data['IO_PO check2'] > data['Ship Date']) & (data['PD_PO'].isnull()) & ~(data['Supply Type'].isin(sply_type))

# condtn2 = data['IO/PO No.'].notnull() & data['PD_PO'].isnull() & (data['Supply Type'].isin(sply_type2)) & \
#             (data['current_date'] < data['IO BOOKED DATE']+timedelta(days=5) )
# condtn3 = 'intransit'
# data['_IO_Status'] = np.where(condtn1a | condtn1b, 'IO SSD Late', \
#                              np.where(condtn2, 'PO PD Null',''))
# data.drop(columns={'current_date'}, inplace=True)

In [34]:
# Scrubbing
data['Region'] = data['Region'].str.upper()
if data['LOB'].dtype == object:
    data['LOB'] = data['LOB'].str[0:3]
data['LOB'] = np.where((data['Products']=='SCO') & (data['LOB']=='FIN'), '', data['LOB'])
data['LOB Prod'] = np.where(data['Products'].isin(['ATM','PC Core']),'Bank','Com') 
data['LOB Prod'] = np.where(data['LOB Prod'].isnull(),'Com',data['LOB Prod'])

In [35]:
data.drop(columns={'current_date'}, inplace=True)

In [36]:
# Hari/Ramiya
# Concatenate Default	Sea	Land	Air
# data['Transit_Days'] = ''

In [37]:
#including Dataref corresponding to current date.
file_name = file[7:].lower()
data['Dataref'] = str(d) + ' ' + file_name 

In [38]:
#Updaing Offer pfbase don Pit type: 20-02-2023
data['Offer PF'] = np.where(data['PID Type'].isin(['K']),'kits',data['Offer PF']) 
data['Offer PF'] = np.where(data['PID Type'].isin(['C']),'Cables',data['Offer PF']) 
data['Offer PF'] = np.where(data['PID Type'].isin(['CPP']),'CPP',data['Offer PF'])
data['Offer PF'] = np.where(data['PID Type'].isin(['IPP']),'IPP',data['Offer PF'])
data['Offer PF'] = np.where(data['PID Type'].isin(['RAW']),'RAW',data['Offer PF'])
data['Products'] = np.where(data['Products'].isnull(),'Other',data['Products'])
data['Offer PF'] = np.where(data['Offer PF'].isnull(),'Other',data['Offer PF'])

In [39]:
# Updated validation flag and Supply Type 
data['Validation Flag'] = data['Validation Flag'].replace({'On hand - RS' : 'OH-RS','On hand':'OH', 'PLAN':'Plan', 'InTransit':'IT', \
                                                          'PD OK':'PD OK', 'WIP/Planned Order':'WIP/Plnd Ord', 'New/Release PO':'New/Rel PO',\
                                                          'Scheduled':'Schd', 'Flow':'Flow', 'Purchase requisition':'Pur Req', 'Planned order':'Plnd Ord',\
                                                          'Shipment in receiving':'In Rcvng', 'Past Due PD':'Past PD', 'WIP/PlannedOrder':'WIP/Plnd Ord'})

data['Supply Type'] = data['Supply Type'].replace({'On hand - RS' : 'OH-RS','On hand':'OH', 'Planned Buy':'Plnd-Buy', 'InTransit':'IT', \
                                                          'PD OK':'PD OK', 'WIP/Planned Order':'WIP/Plnd Ord', 'New/Release PO':'New/Rel PO',\
                                                          'Scheduled':'Schd', 'Flow':'Flow', 'Purchase requisition':'Pur Req', 'Planned order':'Plnd Ord',\
                                                          'Shipment in receiving':'In Rcvng', 'Past Due PD':'Past PD', 'WIP/PlannedOrder':'WIP/Plnd Ord', 'Planned Make':'Plnd-Make', 'Planned Buy':'Plnd-Buy'})

#Update by ramya
cond= data['Validation Flag'].isin(['']) & data['Supply Type'].isin(['PO'])
data['Validation Flag'] = np.where(cond,'PO',data['Validation Flag'])

#updated LOB 
data['LOB'] = np.where(data['LOB'].isin(['CUS']),'',data['LOB'])
data['LOB'] = np.where(data['LOB'].isin(['HSR']),'HOSP',data['LOB'])

#Update Demand
data['Demand'] = np.where(data['Demand'].isin(['Order Int']),'Ord Int',data['Demand'])
data['Demand'] = np.where(data['Demand'].isin(['Order Pre Bld']),'PBO',data['Demand'])
data['Demand'] = np.where(data['Demand'].isin(['Order Flow Stk']),'Flow Stk',data['Demand'])

conditions = [data['Inv Trigger']=='Shipment',data['Inv Trigger']=='Delivery',data['Inv Trigger']=='Other', \
              data['Inv Trigger']=='Installation',data['Inv Trigger']=='Customer Certification']
choices = ['Ship','Del','Oth','Inst','Cert']
data['Inv Trigger'] = np.select(conditions, choices)
data['Inv Trigger'] = np.where (data['Inv Trigger'].astype(str)=='0','',data['Inv Trigger'])

In [40]:
# Calc data['CTS']
# Calc: considering ATS status and OH qty by order level)
# For one order, If "SO Avail to Ship"==OK/Yes; and Check enough OH/RS for rest of the lines --> then CTS
df = data.copy()
df = df[['DEMAND VAL','OH Qty','Res Qty','Supply Qty']]
df = df.groupby(['DEMAND VAL'])['OH Qty','Res Qty','Supply Qty'].sum().reset_index()
df['total'] = df['OH Qty'] + df['Res Qty']
df['CTS'] = np.where(df['total'] > df['Supply Qty'],'CTS','No CTS')
data = pd.merge(data, df[['DEMAND VAL','CTS']], left_on=['DEMAND VAL'], right_on=['DEMAND VAL'],how='left',validate='m:1')

data['Dmd Cnt'] =  1 / data.groupby('DEMAND VAL')['DEMAND VAL'].transform('count')

C:\Users\vv185114\AppData\Local\Temp\3\ipykernel_10908\3332769156.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df = df.groupby(['DEMAND VAL'])['OH Qty','Res Qty','Supply Qty'].sum().reset_index()


In [41]:
#CPM names

cpm_names = pd.read_csv(r'E:\_Projects\orders\rcv\\CPM_Names.csv',encoding='utf-8', low_memory=False)
cpm_names.rename(columns = {'Customer Number': 'Master Customer Number','CPM Name 2023':'CPM','Manager Name 2023':'CPM Mngr'}, inplace = True)
cpm_names.drop_duplicates(inplace=True,subset=['Master Customer Number'],keep='first')
data = data.merge(cpm_names[['Master Customer Number','CPM','CPM Mngr']],on='Master Customer Number',how='left',validate='m:1')

In [42]:

#comments
path = r'E:\_Projects\workbench'
comments =  pd.read_csv(path + r'\comments_log\comments_unique.csv', encoding='utf-8', low_memory=False)
#read wb_cols
col_map = pd.read_excel(path + r'\map\wb_cols.xlsx', sheet_name='field_settings_ms_STD', engine='openpyxl')

#iid columns
col_iid_list = col_map[col_map['Key ID'].isin(['IID'])]
col_iid_list = ['IID'] + col_iid_list['Field Name'].tolist()
comments_iid = comments[col_iid_list]
comments_iid = comments_iid[comments_iid['IID'].notnull()]

#uid columns
col_uid_list = col_map[col_map['Key ID'].isin(['UID'])]
col_uid_list = ['UID'] + col_uid_list['Field Name'].tolist()
comments_uid = comments[col_uid_list]
comments_uid = comments_uid[comments_uid['UID'].notnull()]

#merge uiid and iid
data = data.merge(comments_iid, left_on='IID', right_on = 'IID', how='left',validate='m:1')
data = data.merge(comments_uid, left_on='UID', right_on = 'UID', how='left',validate='m:1')

In [43]:
# Calculate DMD SSD and SPLY SSD
data['Calc SO SSD']=''
cond1 = (data['Whse']==data['SO Ship Org'])
cond2 = (data['Whse'].isin(['CHE','CHS','GF1','US','USL','UF2','EPL','EPI']))
data['Calc SO SSD']=np.where(cond1 & cond2, data['Ship Date'],data['Calc SO SSD'])
data['Calc SO SSD']=np.where((~cond1) & cond2, data['Ship Date'] - pd.to_timedelta(data['Transit_recom_days'], unit='D'), \
                              data['Calc SO SSD'])
align_past('Calc SO SSD')
# data['Calc SO SSD'] = data['final']
data.drop(columns={'Previous_week_Date'}, inplace=True)

data['SO SSD Override++'] = pd.to_datetime(data['SO SSD Override++'],errors='coerce')
data['Calc SO SSD Final'] = np.where(data['SO SSD Override++'].notnull(), data['SO SSD Override++'], \
                                np.where(data['Calc SO SSD'].notnull(), data['Calc SO SSD'], data['Ship Date']))
#----------------------------------------------------------------------------#

# data['Calc Sply SSD']=''
# data['temp'] = pd.to_datetime('today')
# cond3 = (data['Supply Type'].isin(['OH','OH-RS']))
# cond4 = (data['Supply Type'].isin(['PD']))
# data['Calc Sply SSD'] = np.where(cond2 & cond3, data['temp'] - timedelta(days=14), data['Calc Sply SSD'])
# data['Calc Sply SSD'] = np.where(cond2 & cond4, data['PD_PO'] + pd.to_timedelta(data['Transit_recom_days'], unit='D'),\
#                                                                                          data['Calc Sply SSD'])
# data['Calc Sply SSD'] = data['Calc Sply SSD'].astype('datetime64[ns]')
# align_past('Calc Sply SSD')
# data['Calc Sply SSD'] = data['final']
# data.drop(columns={'final'}, inplace=True)

# data.drop(columns={'temp'}, inplace=True)

e:\softwares\anaconda3\Lib\site-packages\pandas\core\arrays\timedeltas.py:908: RuntimeWarning: invalid value encountered in cast
  base = data.astype(np.int64)
e:\softwares\anaconda3\Lib\site-packages\pandas\core\arrays\timedeltas.py:912: RuntimeWarning: invalid value encountered in cast
  data = (base * m + (frac * m).astype(np.int64)).view("timedelta64[ns]")


In [44]:
#  data['Open Qty'] = data['Open Qty'].astype('Int64')
# data['Open Qty'] = data['Open Qty'].values.astype(np.int64)
# data['Res Qty'] = data['Res Qty'].values.astype(np.int64)
# data['OH Qty'] = data['OH Qty'].values.astype(np.int64)
# data['Supply Qty'] = data['Supply Qty'].values.astype(np.int64)
# data['Qty'] = data['Supply Qty'].values.astype(np.int64)
# data['Supply Qty'].head(10)
 

In [45]:
#Change dtypes str to qty
# col_list=['Open Qty','Res Qty','OH Qty','Supply Qty','Qty']
# for col in col_list:
#     data[col] = data[col].values.astype(np.int64)

In [46]:
# import pandas as pd
# Status on IO: Late
race_df  =  pd.read_excel(r'E:\_Projects\lookups\RACE_IO_SSD.xlsx', engine='openpyxl')
race_df.sort_values(by=['Actual Ship Date'], ascending = True,inplace = True)
race_df.drop_duplicates(subset=['Order Number'],keep='last',inplace=True)
data = data.merge(race_df[['Order Number','Actual Ship Date']],left_on='IO No.',right_on='Order Number',how='left',validate='m:1')
data['Actual Ship Date'] = np.where(data['Actual Ship Date'].isnull(),data['IO SSD'],data['Actual Ship Date'])
data['Actual Ship Date']  = data['Actual Ship Date'] + pd.to_timedelta(data['Transit_recom_days'], unit='D')
data['Plant SSD Trans'] =  data['Plant SSD'] + pd.to_timedelta(data['Transit_recom_days'], unit='D')


cond = (data['Plant SSD'].isnull() | data['Validation Flag'].isin(['Plan']))
logic_1= (data['Whse'] == data['SO Ship Org']) & (data['Plant SSD']>=data['Ship Date'])
logic_2 = (data['Whse'] != data['SO Ship Org']) & (data['Supply Type']=='IT') & (data['Actual Ship Date'] >=data['Ship Date'])
logic_3 = (data['Whse'] != data['SO Ship Org']) & ( data['Plant SSD Trans'] >=data['Ship Date']) & (data['Supply Type']!='IT')

conds = [~cond & logic_1,~cond & logic_2,~cond & logic_3]
choices = ['Late','Late','Late']

data['_IO_Status'] = np.select(condlist=conds, choicelist=choices)
data['_IO_Status']  = np.where(data['_IO_Status']=='0','',data['_IO_Status'])
data.drop(columns={'Plant SSD Trans','Actual Ship Date','Order Number'}, axis=1, inplace=True) 

e:\softwares\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
e:\softwares\anaconda3\Lib\site-packages\pandas\core\arrays\timedeltas.py:908: RuntimeWarning: invalid value encountered in cast
  base = data.astype(np.int64)
e:\softwares\anaconda3\Lib\site-packages\pandas\core\arrays\timedeltas.py:912: RuntimeWarning: invalid value encountered in cast
  data = (base * m + (frac * m).astype(np.int64)).view("timedelta64[ns]")


In [47]:
col_map = pd.read_excel(r'E:\_Projects\workbench\map\wb_cols.xlsx', sheet_name='field_list', engine='openpyxl')
col_map = col_map['Fields'].unique().tolist()
data = data[col_map]

In [48]:
#remove enter in all columns
data = data.replace('\n',';', regex=True)

In [49]:
# export wb data into local folder
out_name = 'workbench_data_' + file[7:].lower() + \
            '_' + str(datetime.strptime(d, '%d-%b-%Y').year).zfill(4) + str(datetime.strptime(d, '%d-%b-%Y').month).zfill(2) + \
            str(datetime.strptime(d, '%d-%b-%Y').day).zfill(2)

out_name_wPO = 'workbench_data_' + file[7:].lower() + \
            '_wPO_' + str(datetime.strptime(d, '%d-%b-%Y').year).zfill(4) + str(datetime.strptime(d, '%d-%b-%Y').month).zfill(2) + \
            str(datetime.strptime(d, '%d-%b-%Y').day).zfill(2)


#save local in csv
dst_local = r'E:\_Projects\_outputs\orders\snd\\' 
data.to_csv(dst_local + out_name + '_v2.csv', index=False)

#save server tool in csv
dst_tool = r'E:\_Projects\workbench\snd\\'
data.to_csv(dst_tool + out_name + '.zip', compression={'method': 'zip', 'archive_name': out_name + '.csv'}, index=False)
#input()
sharpoint_Path =r'\\wtc1501cifs.prod.local\ORGSHARE\TEAMS\ERP Shared Folder\NCR Atleos\Atleos Operations Planning\Production\workbench\snd\\'
data.to_csv(sharpoint_Path + out_name + '.zip', compression={'method': 'zip', 'archive_name': out_name + '.csv'}, index=False)



In [50]:
# CAll allocation script
file_name = file[7:].lower()
if file_name == 'eumocp':
    %run Orders_Allocation.ipynb
    #%run alloc_v2.py enn eumocp 20240514
    #%run alloc_v2.py jab eumocp 20240514

site_name: enn; data: eumocp_20240301


FileNotFoundError: [Errno 2] No such file or directory: 'E:\\_Projects\\_outputs\\orders\\snd\\workbench_data_eumocp_20240301_v2.csv'

site_name: jab; data: eumocp_20240301


FileNotFoundError: [Errno 2] No such file or directory: 'E:\\_Projects\\_outputs\\orders\\snd\\workbench_data_eumocp_20240301_v2.csv'

In [51]:
today_date = datetime.now(pytz.timezone('Asia/Calcutta'))
today_date = today_date.strftime('%Y%m%d')
pervious_date='20240115'
pervious_date_enn='20231225'
pervious_date_USI='20230828'
print(today_date)


20240301


In [52]:
if file_name =='eumocp':
    %run Orders.ipynb
    if PD_Enn ==0:
        enn_alloc= pd.read_csv(r'E:\_Projects\_outputs\orders\snd\wb_enn_allocation_'+today_date+'.csv',encoding='utf-8', low_memory=False, keep_default_na='')
        enn_alloc['Plant'] = 'EN2'
        enn_alloc['Demand'] = 'PO-ENN'
    else:
        enn_alloc= pd.read_csv(r'E:\_Projects\_outputs\orders\snd\wb_enn_allocation_'+pervious_date_ENN+'.csv',encoding='utf-8', low_memory=False, keep_default_na='')
        enn_alloc['Plant'] = 'EN2'
        enn_alloc['Demand'] = 'PO-ENN'
    if PD_Jab == 0:    
        jab_alloc= pd.read_csv(r'E:\_Projects\_outputs\orders\snd\wb_jab_allocation_'+today_date+'.csv',encoding='utf-8', low_memory=False, keep_default_na='')
        jab_alloc['Plant'] = 'JB2'
        jab_alloc['Demand'] = 'PO-JAB'
    else:
        jab_alloc= pd.read_csv(r'E:\_Projects\_outputs\orders\snd\wb_jab_allocation_'+pervious_date_JAB+'.csv',encoding='utf-8', low_memory=False, keep_default_na='')
        jab_alloc['Plant'] = 'JB2'
        jab_alloc['Demand'] = 'PO-JAB'

    usi_alloc= pd.read_csv(r'E:\_Projects\_outputs\orders\snd\wb_usi_allocation_'+pervious_date_USI+'.csv',encoding='utf-8', low_memory=False, keep_default_na='')
    usi_alloc['Plant'] = 'UN2'
    usi_alloc['Demand'] = 'PO-USI'

    alloc = pd.concat([enn_alloc,jab_alloc,usi_alloc],axis=0)
    alloc = alloc[data.columns]

    set_date_frmt = ['Booked Dt','Request Dt','PD_Order','Ship Date','SAD','IO BOOKED DATE','IO SSD','IO SAD','PD_PO','CRSD','PRD','Plant Ship date','NEED BY DATE','Plant SSD']
    for field in set_date_frmt:
        alloc[field] = pd.to_datetime(alloc[field])

    data = pd.concat([data,alloc],axis=0)

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\_Projects\\_outputs\\orders\\snd\\wb_enn_allocation_20240301.csv'

In [ ]:

dst_local = 'E:\_Projects\_outputs\orders\snd\\'
data.to_csv(dst_local + out_name_wPO + '_v2.csv', index=False)

# dst_dump = r'\\dayorg1\orgshare\TEAMS\ERP Shared Folder\Global S&OP\workbench\snd\\'
# data.to_csv(dst_dump + out_name + '.csv', index=False)

# dst_tool = r'\\dayorg1\orgshare\TEAMS\ERP Shared Folder\Global S&OP\Supply Plan\workbench\snd\\'
# data.to_csv(dst_tool + out_name + '.zip', compression={'method': 'zip', 'archive_name': out_name + '.csv'}, index=False)